# Analyzing Political Tweets on a Depression Prediction ML Model
### Sam Spell, James Tipton

Political rhetoric and discussions have seemingly become more polarized recently. In history and while reaching adulthood, being able to vote and be a part of politics is a very important role in a stable and healthy society. This project aims to use machine learning to develop a model to predict depression based on a string of text from twitter. Once this model is developed, it can be used to conduct an analysis on political messages sent online. We will be able to draw out patterns in twitter texts that the machine learning model classifies as showing signs of Depression. Another goal of this machine learning model is to extract patterns of text that can be connected to patterns of political messaging if they exist, and to compare this to a temporal aspect. With the changing view on polarized politics, it will be interesting to test if there is a change in the prevalence of messages classified with “depression” throughout different political times.




#### Step 1: Clean the datasets to prepare for the model
Import libraries

In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


import numpy as np
from numpy import savetxt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier

### Run these downloads once:


In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

### Process Data:

In [4]:
#filter for stopwords

# isolate text column of dataset
d = pd.read_csv("depression.csv")
p = pd.read_csv("COVIDSenti.csv")
#comb = p["Title"].fillna('') +  ' ' + p['Text'].fillna('')
#comb2 = comb
text = d["clean_text"]
review = p['tweet']
review_origin = review

# determine stopwords
stop_words = set(stopwords.words('english'))

# define function to remove stopwords
def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = " ".join(filtered_tokens)
    return filtered_text

text = text.apply(remove_stopwords)
review = review.apply(remove_stopwords)
#comb = comb.apply(remove_stopwords)

#lemmatize and stem each reddit post in the dataset

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = " ".join(lemmatized_tokens)
    return lemmatized_text

text = text.apply(lemmatize_text)
review = review.apply(lemmatize_text)

stemmer = PorterStemmer()

def stem_text(text):
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_text = " ".join(stemmed_tokens)
    return stemmed_text

text = text.apply(stem_text)
review = review.apply(stem_text)
#print(comb.head())
#print(comb2.head())



### Machine Learning Process (RF):

In [5]:
#cleaned text



x_train, x_test, y_train, y_test = train_test_split(text, d['is_depression'], test_size=0.33, random_state=42)

# convert phrases into numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)



# train SVM model
#clf = SVC(kernel='linear', C=1.0)
#clf.fit(x_train, y_train)

clf = RandomForestClassifier(n_estimators=400, random_state=42)
clf.fit(x_train, y_train)


# evaluate SVM model
y_pred = clf.predict(x_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))

confusion_mat = confusion_matrix(y_test, y_pred)

print('Confusion matrix:\n', confusion_mat)




#x_new = vectorizer.transform(new_text)
x_new = vectorizer.transform(review)
y_new_pred = clf.predict(x_new)

print(y_new_pred)

print(f"{(sum(y_new_pred) / len(review)) * 100:.2f}%")

print(len(y_new_pred))
print(sum(y_new_pred))




for name in p['label'].unique():
    print(str(name) + " Count")
    count = 0
    name_cnt = 0
    for i in range(len(y_new_pred)):
        if p.iloc[i, 1] == name:
            name_cnt += 1
            if y_new_pred[i] == 1:
                count += 1
    print()
    print(str(count) + " out of " + str(name_cnt))
    print(f"{(count / name_cnt) * 100:.2f}%")
    print()
    print()


Accuracy: 0.9510188087774295
Precision: 0.9862306368330465
Recall: 0.9131474103585657
F1 score: 0.9482829954489036
Confusion matrix:
 [[1281   16]
 [ 109 1146]]
[0 0 0 ... 0 0 0]
11.76%
90000
10584
neu Count

8080 out of 67385
11.99%


neg Count

1785 out of 16335
10.93%


pos Count

719 out of 6280
11.45%


